In [ ]:

from pymongo import MongoClient
from kafka import KafkaProducer
import json
from bson.json_util import dumps

"""
KAFKA PRODUCER FOR STREAMING VISUALIZATION
As a part of the reuquirement, we need to implement a real-time kafka producer that detect
everytime a new nput is added into our mongoDb violation Colection from the previous streaming proccess we 
implemented previously. This are done through$watch operation in mongoDB on any insert operation being triggered
when that happen, we pass it to the consumer which are connected to the real-time matplotlib visualization
"""

mongo_client = MongoClient("172.27.65.143:27017")
db = mongo_client["fit3182_db"]
collection = db["violations"]

# Kafka producer setup
producer = KafkaProducer(
    bootstrap_servers="172.17.0.1:9092",
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Stream topic for streaming viz
stream_topic = "mongo_plot_stream"

# Start watching for new inserts from DbWriter
try:
    with collection.watch([{"$match": {"operationType": "insert"}}]) as stream:
        print("📡 Streaming new data inserted via DbWriter...")
        for change in stream:
            full_doc = change["fullDocument"]
            producer.send(stream_topic, full_doc)
            print(f"✅ Sent to Kafka ({stream_topic}): {full_doc}")
except Exception as e:
    print("Error:", e)
finally:
    producer.flush()
    producer.close()